In [124]:
import requests
import folium
import collections

In [176]:
app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
params = {
    'query': '호식이두마리치킨 서귀포',
    'page': 1,
    'rect': '',
}
headers  = {
    'Authorization': app_key
}
resp = requests.get(url, params=params, headers=headers)
resp.json()

{'documents': [],
 'meta': {'is_end': True,
  'pageable_count': 0,
  'same_name': {'keyword': '호식이두마리치킨 서귀포',
   'region': [],
   'selected_region': ''},
  'total_count': 0}}

In [158]:
def get_store_list(start_x,start_y,end_x,end_y):
    offset = 0.01
    print(start_x,start_y,end_x,end_y)
    cnt = 1
    resp_list = []
    while True:
        jump_x = 0.5
        jump_y = 0.5
        params = {
            'query': '호식이두마리',
            'page': cnt,
            'rect': f'{start_x-offset},{start_y-offset},{end_x+offset},{end_y+offset}'
        }
        headers  = {
            'Authorization': app_key
        }
        resp = requests.get(url, params=params, headers=headers)
        search_count = resp.json()['meta']['total_count']
        print(search_count)
        if search_count > 45:
            print('big data dividing...')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            # 1
            resp_list.extend(get_store_list(start_x,start_y,dividing_x,dividing_y))
            # 2 x만 이동
            resp_list.extend(get_store_list(dividing_x,start_y,end_x,dividing_y))
            # 3 y만 이동
            resp_list.extend(get_store_list(start_x,dividing_y,dividing_x,end_y))
            # 4 x,y 전부 이동
            resp_list.extend(get_store_list(dividing_x,dividing_y,end_x,end_y))
            return resp_list
        else:
            if resp.json()['meta']['is_end']:
                resp_list.extend(resp.json()['documents'])
                return resp_list        
            else:
                print('paging...')
                cnt += 1
                resp_list.extend(resp.json()['documents'])
                
def getLatLng(addr):
#     addr = '서울특별시 구로구 디지털로 306'
    app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])
    else:
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print('불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)
        
def get_avg_point(x_list, y_list):
    x = sum(x_list) / len(x_list)
    y = sum(y_list) / len(y_list)
    return x, y

def draw_map(addr):
    map_noman = folium.Map(location=get_avg_point(addr_point_list),zoom_start=7)
#     disabled_sup = list(set(df_noman['장애인편의발급기형태']))
#     for x, y in addr_point_list:
#         if df_noman['장애인편의발급기형태'][cnt] == disabled_sup[0]:
#             add_marker(map_noman, x, y)
    return map_noman

def draw_marker(x, y, tel):
    marker = folium.Marker((x, y), popup=tel)
    icon=folium.Icon(color='orange',icon='glyphicon glyphicon-cutlery')
    icon.add_to(marker)
    return marker

In [159]:
app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
start_x = 126 # ~ 129.7, 3.4
store_list = []
for i in range(1,10):
    end_x = start_x + jump_x
    start_y = 33 # ~ 38.3, 4.9
    for j in range(1,13):
        end_y = start_y + jump_y
        store_list_one = get_store_list(start_x,start_y,end_x,end_y)
        store_list.extend(store_list_one)
        start_y = end_y
    start_x = end_x
store_list

126 33 126.5 33.5
5
126 33.5 126.5 34.0
1
126 34.0 126.5 34.5
0
126 34.5 126.5 35.0
5
126 35.0 126.5 35.5
1
126 35.5 126.5 36.0
0
126 36.0 126.5 36.5
0
126 36.5 126.5 37.0
1
126 37.0 126.5 37.5
0
126 37.5 126.5 38.0
1
126 38.0 126.5 38.5
0
126 38.5 126.5 39.0
0
126.5 33 127.0 33.5
3
126.5 33.5 127.0 34.0
6
126.5 34.0 127.0 34.5
0
126.5 34.5 127.0 35.0
2
126.5 35.0 127.0 35.5
32
paging...
32
paging...
32
126.5 35.5 127.0 36.0
8
126.5 36.0 127.0 36.5
1
126.5 36.5 127.0 37.0
5
126.5 37.0 127.0 37.5
82
big data dividing...
126.5 37.0 126.75 37.25
1
126.75 37.0 127.0 37.25
2
126.5 37.25 126.75 37.5
26
paging...
26
126.75 37.25 127.0 37.5
57
big data dividing...
126.75 37.25 126.875 37.375
10
126.875 37.25 127.0 37.375
13
126.75 37.375 126.875 37.5
17
paging...
17
126.875 37.375 127.0 37.5
26
paging...
26
126.5 37.5 127.0 38.0
87
big data dividing...
126.5 37.5 126.75 37.75
24
paging...
24
126.75 37.5 127.0 37.75
64
big data dividing...
126.75 37.5 126.875 37.625
25
paging...
25
126.875 37.5

[{'address_name': '제주특별자치도 제주시 연동 1491-3',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 치킨 > 호식이두마리치킨',
  'distance': '',
  'id': '15220145',
  'phone': '064-773-9922',
  'place_name': '호식이두마리치킨 연동신시가지점',
  'place_url': 'http://place.map.kakao.com/15220145',
  'road_address_name': '제주특별자치도 제주시 국기중길 5-6',
  'x': '126.491685722983',
  'y': '33.4802936485155'},
 {'address_name': '제주특별자치도 제주시 애월읍 애월리 1713-1',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 치킨 > 호식이두마리치킨',
  'distance': '',
  'id': '26902290',
  'phone': '064-799-9909',
  'place_name': '호식이두마리치킨 애월읍내점',
  'place_url': 'http://place.map.kakao.com/26902290',
  'road_address_name': '제주특별자치도 제주시 애월읍 애월로11길 11',
  'x': '126.320324644821',
  'y': '33.465487016673'},
 {'address_name': '제주특별자치도 제주시 노형동 2613-10',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 치킨 > 호식이두마리치킨',
  'distance': '',
  'id': '202

In [164]:
store_list2 = []
store_list = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in store_list)))
for i in store_list:
    j = i['place_name']
    tel = i['phone']
    addr = i['address_name']
    x_list = float(i['y'])
    y_list = float(i['x'])
    store_list2.append({'jijum':j, 'tel':tel, 'addr':addr, 'x':x_list, 'y':y_list})
len(store_list2)

822

In [166]:
x_list = list(map(lambda i:i['x'], store_list2))
y_list = list(map(lambda i:i['y'], store_list2))
tel_list = list(map(lambda i:i['tel'], store_list2))
hosik2_map = folium.Map(location=get_avg_point(x_list, y_list),zoom_start=7)
for x, y, t in zip(x_list, y_list, tel_list):
    marker = draw_marker(x, y, t)
    marker.add_to(hosik2_map)
# hosik2_map    